# Transfer Learning

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

In [2]:
# ONLY FOR NVIDIA
#CUDA_VISIBLE_DEVICES = 0

In [5]:
#tf.config.list_physical_devices('GPU')

In [7]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 69s 0us/step


In [9]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [11]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [13]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [15]:
flowers_images_dict['roses'][:5]

[PosixPath('datasets/flower_photos/roses/4684127262_6c3346188d.jpg'),
 PosixPath('datasets/flower_photos/roses/1540738662_7b4152e344_m.jpg'),
 PosixPath('datasets/flower_photos/roses/7316409504_7cf3707f8a_m.jpg'),
 PosixPath('datasets/flower_photos/roses/3664842094_5fd60ee26b.jpg'),
 PosixPath('datasets/flower_photos/roses/5799616059_0ffda02e54.jpg')]

In [17]:
def resize_images(df):
    X, y = [], []
    for flower_name, images in flowers_images_dict.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img, (224,224))
            X.append(resized_img)
            y.append(flowers_labels_dict[flower_name])
    return X, y

In [19]:
X, y = resize_images(flowers_images_dict)

In [20]:
X = np.array(X)
y = np.array(y)

# Scale/split Data

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [24]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

2752 918
2752 918


In [30]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [34]:
IMAGE_SHAPE = (224,224)
IMAGE_SHAPE+(3,)


(224, 224, 3)

# ANN Model

In [37]:
from sklearn.metrics import classification_report, confusion_matrix

In [39]:
ann = Sequential([
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
ann.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [41]:
ann.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [43]:
ann.fit(X_train, y_train, epochs=5)

Epoch 1/5


2024-08-23 00:39:00.287975: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


86/86 ━━━━━━━━━━━━━━━━━━━━ 22s 237ms/step - accuracy: 0.2804 - loss: 26.4650
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 244ms/step - accuracy: 0.3914 - loss: 4.6085
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 20s 234ms/step - accuracy: 0.4222 - loss: 3.0688
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 20s 227ms/step - accuracy: 0.4621 - loss: 2.4468
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 20s 230ms/step - accuracy: 0.5226 - loss: 1.8405


In [45]:
y_pred = ann.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]
print(classification_report(y_test, y_pred_classes))

 3/29 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

2024-08-23 00:40:45.691301: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552738816 exceeds 10% of free system memory.


29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step
              precision    recall  f1-score   support

           0       0.70      0.04      0.08       176
           1       0.31      0.25      0.28       154
           2       0.43      0.22      0.29       226
           3       0.24      0.93      0.38       150
           4       0.49      0.21      0.29       212

    accuracy                           0.30       918
   macro avg       0.43      0.33      0.26       918
weighted avg       0.44      0.30      0.26       918



# Transfer Learning

In [47]:
#print(hub.version)

In [49]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
import tf_keras

In [51]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224,224,3), trainable=False, 
)

In [59]:
num_of_flowers = 5

tf_keras

model = tf_keras.Sequential([
    pretrained_model_without_top_layer,
    tf_keras.layers.Dense(num_of_flowers, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [61]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [63]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


2024-08-23 00:43:28.549071: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1657012224 exceeds 10% of free system memory.


86/86 [==============================] - 41s 439ms/step - loss: 0.7817 - accuracy: 0.7068
Epoch 2/5
86/86 [==============================] - 36s 423ms/step - loss: 0.3971 - accuracy: 0.8663
Epoch 3/5
86/86 [==============================] - 37s 424ms/step - loss: 0.3067 - accuracy: 0.9023
Epoch 4/5
86/86 [==============================] - 36s 423ms/step - loss: 0.2630 - accuracy: 0.9157
Epoch 5/5
86/86 [==============================] - 37s 427ms/step - loss: 0.2229 - accuracy: 0.9353


In [65]:
model.evaluate(X_test, y_test)

2024-08-23 00:48:50.012591: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552738816 exceeds 10% of free system memory.


29/29 [==============================] - 13s 424ms/step - loss: 0.3701 - accuracy: 0.8660


[0.37014371156692505, 0.8660130500793457]